In [4]:
from pprint import pprint
from Bio import SeqIO
import utils
import subprocess

import random


In [5]:
genuses = utils.get_genuses()


# longest sequences per cluster

In [6]:
def parse_line(line):
    length = int(line.partition("aa")[0].partition("\t")[2])
    species_and_gene = line.partition(">")[2].partition("...")[0]
    species = species_and_gene.partition("|")[0]
    gene = species_and_gene.partition("|")[2]
    accuracy_string = line.partition("...")[2]
    if "*" in accuracy_string:
        accuracy = 100
    else:
        accuracy = float(line.partition("at ")[2].partition("%")[0])
    return [length, species, gene, accuracy]


In [7]:
def get_longest_seq_from_cluster(cluster):
    sorted_list = sorted(cluster, key=lambda x: (-x[0], -x[3]))
    return sorted_list[0]


In [8]:
longest_seqs_in_clusters = list()
for genus in genuses:
    with open(f"../large_data/genuses/{genus}/{genus}_clustered.fasta.clstr") as f:
        cluster = list()
        line = f.readline()
        while line:
            if line.startswith(">"):
                if cluster:
                    longest_seq = get_longest_seq_from_cluster(cluster)
                    longest_seqs_in_clusters.append(longest_seq)
                    cluster = list()
            else:
                cluster.append(parse_line(line))
            line = f.readline()


In [9]:
pprint(longest_seqs_in_clusters[0])
pprint(len(longest_seqs_in_clusters))


[36, 'GB_GCA_003477305.1', 'DMTN01000058.1_80', 100]
2118544


In [10]:
wanted_seqs = set([x[2] for x in longest_seqs_in_clusters])


In [11]:
# output_file = "../large_data/longest_seqs_from_clusters.fasta"
# input_file = "../large_data/all_genus_specific_proteins.fasta"

# with open(output_file, "w") as handle:
#     records = (r for r in SeqIO.parse(input_file, "fasta") if r.id in wanted_seqs)

#     SeqIO.write(records, handle, "fasta")


# homologues

In [12]:
# make a dict containing lists of genes per species
# { species_id: set?[gene_id, gene_id], species_id...}
genes_per_species = dict()
for s in longest_seqs_in_clusters:

    species = s[1]
    gene = s[2]
    if species not in genes_per_species:
        genes_per_species[species] = {
            gene,
        }
    else:
        genes_per_species[species].add(gene)


In [13]:
# homologues_per_species = dict()
# print(len(genes_per_species))
# cnt = 0
# for species in genes_per_species:
#     # species= "RS_GCF_000238995.1"
#     if cnt % 2000 == 0:
#         print(cnt)
#     cnt += 1
#     homologues_per_species[species] = set()
#     gene_ids = genes_per_species[species]
#     input_file = f"../large_data/protein_faa_reps/bacteria/{species}_protein.faa"
#     homologue_ids = list(r.id for r in SeqIO.parse(input_file, "fasta"))

#     if (len(homologue_ids) - len(gene_ids)) < len(gene_ids):
#         print(f"{species} {len(homologue_ids)} {len(gene_ids)} ")
#         for i in range(len(gene_ids)):
#             choice = random.choice(homologue_ids)
#             while choice in homologues_per_species[species]:
#                 choice = random.choice(homologue_ids)
#             homologues_per_species[species].add(choice)
#     else:
#         for i in range(len(gene_ids)):
#             choice = random.choice(homologue_ids)
#             while choice in homologues_per_species[species] or choice in gene_ids:
#                 choice = random.choice(homologue_ids)
#             homologues_per_species[species].add(choice)


In [14]:
# output_file = "../large_data/homologues.fasta"
# with open(output_file, "w") as handle:

#     for species in homologues_per_species:

#         wanted = homologues_per_species[species]
#         input_file = f"../large_data/protein_faa_reps/bacteria/{species}_protein.faa"
#         records = (r for r in SeqIO.parse(input_file, "fasta") if r.id in wanted)
#         count = SeqIO.write(records, handle, "fasta")
#         if count < len(wanted):
#             print("Warning %i IDs not found in %s" % (len(wanted) - count, input_file))


# random sequences

In [15]:
# for species in genes_per_species:
#     input_file = f"../large_data/gtdb_genomes_reps_r202/{species[3:6]}/{species[7:10]}/{species[10:13]}/{species[13:16]}/{species[3:]}_genomic.fna.gz"
#     subprocess.run(["gzip", "-d", input_file])


input_file = "../large_data/homologues.fasta"
records = (r for r in SeqIO.parse(input_file, "fasta"))
for i in records:
    rec = i
    break
rec
input_file = "../large_data/longest_seqs_from_clusters.fasta"
records2 = (r for r in SeqIO.parse(input_file, "fasta"))



In [80]:
len(list((r for r in records2)))

2118544


to find: 20

grep ">" -c large_data/gtdb_genomes_reps_r202/GCA/003/150/015/GCA_003150015.1_genomic.fna
12

grep ">" -c large_data/protein_faa_reps/bacteria/GB_GCA_003150015.1_protein.faa 
1161


In [48]:
species = "GB_GCA_003150015.1"
print(len(genes_per_species[species]))

20


In [47]:

cnt = 0
for species in genes_per_species:
    print(cnt)
    if cnt % 1000 == 0:
        print(cnt)
    cnt+=1
    gene_ids = genes_per_species[species]
    path = f"../large_data/gtdb_genomes_reps_r202/{species[3:6]}/{species[7:10]}/{species[10:13]}/{species[13:16]}/"
    input_file = f"../large_data/gtdb_genomes_reps_r202/{species[3:6]}/{species[7:10]}/{species[10:13]}/{species[13:16]}/{species[3:]}_genomic.fna"
    records = [r for r in SeqIO.parse(input_file, "fasta")]
    taken = list()
    print(len(records))
    print(len(gene_ids))
    print(input_file)
    for i in range(len(gene_ids)):
        output_file = path + str(i) + ".fasta"
        sample = random.sample(records,1)
        while sample[0].id in taken:
            sample = random.sample(records,1)
        taken.append((sample[0].id))
        with open(output_file, "w") as handle:
            SeqIO.write(sample, handle, "fasta")



0
0
71
23
../large_data/gtdb_genomes_reps_r202/GCA/003/477/305/GCA_003477305.1_genomic.fna
1
206
28
../large_data/gtdb_genomes_reps_r202/GCA/903/844/055/GCA_903844055.1_genomic.fna
2
12
20
../large_data/gtdb_genomes_reps_r202/GCA/003/150/015/GCA_003150015.1_genomic.fna


KeyboardInterrupt: 

In [44]:
cnt

3

In [ ]:
# strip numbers after underscore from gene ids
# but now there will be duplicates probli

output_file = "../large_data/randoms.fasta"
with open(output_file, "a") as handle:
    used_ids = set()
    for species in genes_per_species:
        gene_ids = genes_per_species[species]
        # get packed input file by splitting the species string
        # unpack input file
        # parse input file
        # for i in range len gene_ids
        # choose random sequence
        # while random sequence in used_ids
        # choose random sequence
        #  if (len(nt sequences) - len(gene_ids)) >= len(gene_ids):
        # while random sequence id in gene_ids
        # choose random sequence
        # write sequence to text file
        # put the sequence into getorf
        # parse the output file with seqio
        # choose random sequence
        # append sequence to file
